In [25]:
from __future__ import annotations

```{=latex}
\usepackage{hyperref}
\usepackage{graphicx}
\usepackage{listings}
\usepackage{textcomp}
\usepackage{fancyvrb}

\newcommand{\passthrough}[1]{\lstset{mathescape=false}#1\lstset{mathescape=true}}
\providecommand{\tightlist}{%
  \setlength{\itemsep}{0pt}\setlength{\parskip}{0pt}}

```

```{=latex}
\title{Iterate, Iterate, Iterate}
\author{Moshe Zadka -- https://cobordism.com}
\date{}

\begin{document}
\begin{titlepage}
\maketitle
\end{titlepage}
```

```{=latex}
\frame{\titlepage}
```

## Iteration protocol

### Iterable vs. Iterator

In [26]:
from typing import TypeVar, Protocol, Any, Generic
from dataclasses import dataclass

T = TypeVar("T")


```{=latex}
\begin{frame}[fragile]
\frametitle{Iterable or Iterator}
```

In [27]:
class Iterable(Protocol[T]):
    def __iter__(self) -> Iterator[T]:
        ...

```{=latex}
\pause
```

In [28]:
class Iterator(Iterable[T]):
    def __next__(self) -> T:
        """Raises StopIteration"""

```{=latex}
\pause

The \verb|__iter__| method returns \verb|self|
```

```{=latex}
\end{frame}
```

When it comes to iterators,
there are two related
"protocols"
that are important to understand.
They refer to each other,
so they have to be explained as a pair.

The first is the
*iterable*
protocol.
This protocol has one method:
`__iter__`.
This method is supposed to return an iterator.
The
*iterator*
protocol
specializes the iterable protocol.
It has another method,
`__next__`,
which returns the next 
item
or raises
`StopIteration`.
On an iterator,
the
`__iter__`
method returns
`self`.


### `iter`

```{=latex}
\begin{frame}[fragile]
\frametitle{Functions}
```

In [31]:
def iter(thing):
    return thing.__iter__()

def next(thing):
    return thing.__next__()
    

```{=latex}
\end{frame}
```

In [32]:
del iter
del next

The
`iter`
function is the right way to get at an object's
`__iter__`.
The real function is slightly more complicated,
since it can call native objects'
iteration slot.

### `__iter__`

```{=latex}
\begin{frame}[fragile]
\frametitle{Making it iterable}
```

In [33]:
@dataclass(frozen=True)
class TwoInts:
    thing_1: int
    thing_2: int
    
    def __iter__(self) -> Iterator[T]:
        return iter([self.thing_1, self.thing_2])

```{=latex}
\end{frame}
```

One way to implement the
`__iter__`
method is to return an iterator
of a different object
which gets the same things.
This is sometimes the correct thing to do,
although not always.

As a teaching example,
this works well!

```{=latex}
\begin{frame}[fragile]
\frametitle{Making it iterable}
```

In [34]:
dec = TwoInts(1, 10)
iter_dec = iter(dec)
print("thing_1", next(iter_dec))
print("thing_2", next(iter_dec))

thing_1 1
thing_2 10


```{=latex}
\end{frame}
```

This is one way to use an iterable object.
Call
`iter()`,
and then call
`next()`
repeatedly on the result.

### `__next__`

```{=latex}
\begin{frame}[fragile]
\frametitle{Moving Forward}
```

In [36]:
class TwoIntIter:
    def __init__(self) -> None:
        self.things = [1, 10]
    
    def __next__(self) -> int:
        try:
            return self.things.pop(0)
        except IndexError:
            raise StopIteration() from None

```{=latex}
\end{frame}
```

When writing a fresh iterator,
it has to be
*stateful*.
Each
`next()`
call,
in general,
returns a different item.
In this example,
the state is the list itself.

```{=latex}
\begin{frame}[fragile]
\frametitle{Moving Forward}
```

In [37]:
iter_dec = TwoIntIter()
print(next(iter_dec))
print(next(iter_dec))

1
10


```{=latex}
\end{frame}
```

In this case,
there is no need
(although it is still allowed)
to call
`iter`.
The rest of the code is the same as the last example.

### `next`

```{=latex}
\begin{frame}[fragile]
\frametitle{Moving Forward: Usage}
```

In [28]:
iter_dec = iter([1, 10])
print(next(iter_dec))
print(next(iter_dec))
try:
    next(iter_dec)
except Exception as exc:
    print("Stopped", repr(exc))

1
10
Stopped StopIteration()


```{=latex}
\end{frame}
```

In more realistic examples of iteration,
we do not know in advance the number of items.
The
`StopIteration` exception has to be caught.

### `for`

```{=latex}
\begin{frame}[fragile]
\frametitle{for: Using iterators}
```

In [35]:
class TwoIntIter:
    def __init__(self) -> None:
        self.things = [1, 10]
    
    def __iter__(self) -> Iterator[int]:
        return self

    def __next__(self) -> int:
        try:
            ret_value = self.things.pop(0)
        except IndexError:
            print("__next__: End of iteration")
            raise StopIteration() from None
        print("__next__: Returning", ret_value)
        return ret_value

```{=latex}
\end{frame}
```

In order to show how
`for`
uses iterator,
this iterator example focuses
on a lot of side effects --
printing.
This is generally not a good idea.
In this specific case,
it helps illustrate when the function gets called.

```{=latex}
\begin{frame}[fragile]
\frametitle{for: Using iterators}
```

In [42]:
iter_dec = TwoIntIter()
print("for: Before")
for val in iter_dec:
    print("for: Got", val)
print("for: After")

for: Before
__next__: Returning 1
for: Got 1
__next__: Returning 10
for: Got 10
__next__: End of iteration
for: After


```{=latex}
\end{frame}
```

Note thatin this example,
the iterator is called before each iteration.
When the exception is raised,
the for loop ends.

### Sequence constructors

```{=latex}
\begin{frame}[fragile]
\frametitle{Sequence constructors: Using iterators}
```

In [38]:
res = list(TwoIntIter())
print("Result is", res)

__next__: Returning 1
__next__: Returning 10
__next__: End of iteration
Result is [1, 10]


```{=latex}
\end{frame}
```

Another common way to consume operators is
via the constructor for sequences.
These constructors will consume the items
in this iterator and construct a sequence from them.

This is a useful way to
"buck-stop"
the iterator.
After constructing the sequence,
the iterator logic is no longer involved.

## Generators

### Generator vs. Iterator

```{=latex}
\begin{frame}[fragile]
\frametitle{Generator vs. Iterator}

Iterator: Protocol \pause

Annoying to implement (state) \pause

Generator: Mechanism to build iterators

\end{frame}
```

The iterator protocol is useful.
It is also non-trivial to implement well.
Because it has to keep track of state,
it can be easy to get messy with it.

One way to avoid this need for manually managing the
state of the iteration
is to use generators.
In generators,
program state is the iterator state.

### `yield`

```{=latex}
\begin{frame}[fragile]
\frametitle{Generators and yield}
```

In [39]:
def two_int_iter():
    print("Begin")
    yield 1
    print("Middle")
    yield 10
    print("End")

```{=latex}
\end{frame}
```

This generators,
except for the print statements,
is similar to the last one.
Note that in this case,
there is no explicit storage of a
2-element list.

```{=latex}
\begin{frame}[fragile]
\frametitle{Generators and yield}
```

In [43]:
iter_dec = two_int_iter()
print("for: Before")
for val in iter_dec:
    print("for: Got", val)
print("for: After")

for: Before
Begin
for: Got 1
Middle
for: Got 10
End
for: After


```{=latex}
\end{frame}
```

When consuming it with
`for`,
it is possible to see exactly when the generator resumes execution.

### `yield from`

```{=latex}
\begin{frame}[fragile]
\frametitle{yield from}
```

In [44]:
def three_int_iter():
    yield from two_int_iter()
    yield 100

list(three_int_iter())

Begin
Middle
End


[1, 10, 100]

```{=latex}
\end{frame}
```

In a generator,
`yield from`
is a way to reuse any
iterator.
The iterator being reused can be
another generator
or not.

### `deque(maxlen=0)`

```{=latex}
\begin{frame}[fragile]
\frametitle{Mindless consumption}
```

In [45]:
from collections import deque

deque(two_int_iter(), maxlen=0)

Begin
Middle
End


deque([], maxlen=0)

```{=latex}
\end{frame}
```

Because generators have side-effects,
sometimes we need only the side effects.
A sequence constructor for doing this is
`deque(maxlen=0)`.
It consumes a generator without storing any elements.

## Combining iterators

```{=latex}
\begin{frame}[fragile]
\frametitle{Iterable to Iterator}

\pause

Input: any iterable

\pause

Output: on-demand iterator

\end{frame}
```

One of the most useful thing about iterators is the concept of
"iterator algebra".
There are many functions that take in an iterable,
or several iterables,
and return iterators.
Since iterators are also iterables,
the result of one function can be fed to others.

Much like in
"numeric algebra",
where a few operations that work on elements and return elements of
the same type,
this idea allows breaking up code into multiple,
more easily digestible,
parts.
Moreover,
again like in
"numeric algebra",
Python comes with useful operations out of the box.

### `enumerate`

```{=latex}
\begin{frame}[fragile]
\frametitle{enumerate}
```

In [49]:
list(enumerate(iter([1, 10, 100])))

[(0, 1), (1, 10), (2, 100)]

```{=latex}
\end{frame}
```

The function
`enumerate`
takes an iterable and returns an iterator
that yields a count as well as the original value.
This is often useful for indexing or other purposes.

### `map`

```{=latex}
\begin{frame}[fragile]
\frametitle{map}
```

In [50]:
from operator import add
from functools import partial

list(map(
    partial(add, 1),
    iter([10, 20]),
))

[11, 21]

```{=latex}
\end{frame}
```

The map function takes some iterables
and applies a function to the items one-by-one.
This is useful to post-process results from iterators.

### `filter`

```{=latex}
\begin{frame}[fragile]
\frametitle{filter}
```

In [53]:
from operator import ge
from functools import partial

list(filter(
    partial(ge, 5),
    iter([1, 10, 3]),
))

[1, 3]

```{=latex}
\end{frame}
```

The
`filter`
function
takes an iterable and returns an iterator
with only the elements that satisfy a condition.
It does
*not*
modify the elements.
It is sometimes used in combination with
`map`.

### `zip`

```{=latex}
\begin{frame}[fragile]
\frametitle{zip}
```

In [54]:
list(zip(
    iter([10, 100]),
    iter([2, 4]),
))

[(10, 2), (100, 4)]

```{=latex}
\end{frame}
```

The
`zip`
function takes iterables
and returns an iterator
which yields tuples --
one from each iterable.


### Comprehensions

```{=latex}
\begin{frame}[fragile]
\frametitle{zip}
```

In [56]:
double_plus_one = (
    2 * a + 1
    for a in
    iter([1, 10])
)

list(double_plus_one)

[3, 21]

```{=latex}
\end{frame}
```

For artbitrary code,
generator comprehensions can be used.
They allow filtering
(with if),
looping over multiple things,
and more.
They are often useful together with
`zip`.

## `itertools`

In [57]:
import itertools

```{=latex}
\begin{frame}[fragile]
\frametitle{itertools}

\pause

standard

\pause

Advanced iterator algebra

\end{frame}
```

Beyond the built-in functions and syntax,
the
`iteratools`
module provides some more useful iteration algebra primitives.
There are more than can be easily covered,
but here is a small taste.

### `chain`

```{=latex}
\begin{frame}[fragile]
\frametitle{chain}
```

In [58]:
list(
    itertools.chain(
        iter([1, 2, 3]),
        iter([4, 5]),
    )
)

[1, 2, 3, 4, 5]

```{=latex}
\end{frame}
```

The
`itertools.chain`
function is similar to sequence addition.
It returns an iterator which yields elements
from the first iterable,
and then the second.
It can take any number of iterables.

### `islice`

```{=latex}
\begin{frame}[fragile]
\frametitle{islice}
```

In [60]:
list(itertools.islice(two_int_iter(), 0, 1))

Begin


[1]

```{=latex}
\end{frame}
```

The
`islice`
function is similar to sequence slicing.
It supports first element, last element,
and step.
Note that because it uses iterators,
it will
*not*
consume all elements.
This makes it particularly useful to cap the length
of long
(or infinite)
iterators.

### `count`

```{=latex}
\begin{frame}[fragile]
\frametitle{count}
```

In [62]:
list(itertools.islice(itertools.count(), 0, 5))

[0, 1, 2, 3, 4]

```{=latex}
\end{frame}
```

The
`count`
function returns a count.
This is an infinite count,
but can be cut down with
`islice`.

## `more_itertools`

In [2]:
import more_itertools

```{=latex}
\begin{frame}[fragile]
\frametitle{more-itertools}

\pause

When itertools is not enough

\end{frame}
```

There are also useful third-party libraries with even more useful
iterator algebra primitives.
One of them has the unassuming name
`more_itertools`.
It contains quite a few useful functions.
Once again,
this is a small taste.

### `chunked`

```{=latex}
\begin{frame}[fragile]
\frametitle{chunked}
```

In [5]:
list(more_itertools.chunked(iter([0, 1, 2, 3]), 2))

[[0, 1], [2, 3]]

```{=latex}
\end{frame}
```

The
`chuncked`
function is useful for batching elements from an iterator.
For example,
this is useful for a remote API that takes several items in a batch.
Without much code modification,
you can tune the size of the batch.

### `distribute`

```{=latex}
\begin{frame}[fragile]
\frametitle{distribute}
```

In [7]:
evens, odds = more_itertools.distribute(
    2,
    iter([0, 1, 2, 3]),
)
print(list(evens))
print(list(odds))

[0, 2]
[1, 3]


```{=latex}
\end{frame}
```

The
`distribute`
function distributes elements between several iterators.
This can be used for parallel processing,
for example.

### `peekable`

```{=latex}
\begin{frame}[fragile]
\frametitle{peekable}
```

In [11]:
some_nums = more_itertools.peekable(
    iter([0, 1, 2, 3, 4])
)

In [12]:
for x in some_nums:
    if some_nums.peek() == 3:
        break
    print(x)

0
1


```{=latex}
\end{frame}
```

Sometimes it's useful to be able to know
"what comes next".
This can be useful when processing a stream,
to only process as much as needed,
and leave the stream in a good state for the next consumer.

### `windowed`

```{=latex}
\begin{frame}[fragile]
\frametitle{windowed}
```

In [19]:
samples = more_itertools.windowed(
    iter([0, 3, 7, 10, 12]),
    2
)

In [20]:
for start, end in samples:
    print(end - start)

3
4
3
2


```{=latex}
\end{frame}
```

The
`windowed`
function gives a moving window over the iterator.
As in the example shown,
it can be used to get the difference between
successive samples.

## Summary

### Iterators and Generators

```{=latex}
\begin{frame}[fragile]
\frametitle{Iterators and Generators}

\pause

Iterators: Useful interface

\pause

Generators: Useful way to implement interface

\end{frame}
```

Iterators provide a useful common interface for
just-in-time execution.
Generators are a useful way to write iterators.

### Algebra for Just-in-time

```{=latex}
\begin{frame}[fragile]
\frametitle{Algebra of Just-in-time}

\pause

Functions that accept and return iterators

\pause

Composable just-in-time tools

\end{frame}
```

Part of the usefulness of the iterator concept
is that it lends itself to an algebra.
This allows expressing abstract computation that
needs to happen step by step
while still supporting composability.

### Ecosystem

```{=latex}
\begin{frame}[fragile]
\frametitle{Ecosystem}

\pause

Research

\pause

Consume

\pause

Contribute

\end{frame}
```

This useful abstraction,
together with Python's popularity,
means there is a big ecosystem.
Research to learn what things you can get
"for free".
Consume them as you would any other PyPI dependency.
If you do find a useful abstraction that's missing,
you can explore contributing it to an existing PyPI package
or upload your own.

```{=latex}
\end{document}
```